# Broadcast NFT on-chain

In [1]:
pip install ecdsa


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from tx_engine import Wallet

from tcp_utils import execute_swap, spend_zkp_to_output, p2pk_to_groth16
from tx_engine_utils import setup_network_connection, tx_from_id, p2pk, spend_p2pkh

# Setup

To run the notebook, you need to setup a proving/verifying key pair. To do so, execute the following command from inside the folder `../nft_proof_system`:

`cargo run --release -- --setup --file ./configs/config_base/setup.toml`

This will generate a proving/verifying key pair for transaction chains at index `0`. See also the README of `nft_proof_systems` for more information.

# Initiate blockchain deployment

In [ ]:
connection = setup_network_connection("testnet")

If you have some public keys, you can use them below. Otherwise, generate new ones using the command

```python
pub_key = Wallet.generate_keypair("BSV_Testnet")
```

In [ ]:
# The public key initial holding the BSV
bsv_pub_key = Wallet.from_hexstr("BSV_Testnet","")
bsv_pub_key.get_address()

In [ ]:
# The public key initially holding the token
token_pub_key = Wallet.from_hexstr("BSV_Testnet","")
token_pub_key.get_address()

In [ ]:
# Directories where we fetch the data
vk_path = "../nft_proof_system/data/keys/help_vk.bin"
proof_path = "../nft_proof_system/data/proofs/"
processed_input_path = "../nft_proof_system/data/proofs/"

#### Turn funding into p2pk

First of all, we need to get some funding. You can use a Faucet as [scrypt.io/faucet](https://scrypt.io/faucet). Then, we need to turn the funding into p2pk because our zkps assume the UTXOs are locked with this type of locking script.

Get two funding UTXO for `token_pub_key` and two for `bsv_pub_key`. Then, paste their txids below (in the cell below the one for `token_pub_key`, later on the ones for `bsv_pub_key`).

In [ ]:
token_funding_tx = [
    tx_from_id(txid, connection) for txid in [
    "",
    "",
    ]
]

token_funding_ixs = [0,0]

In [ ]:
# Generate genesis token and send funds to bsv_pub_key
amount = sum([tx.tx_outs[index].amount for (tx, index) in zip(token_funding_tx, token_funding_ixs)])

outputs = [
    p2pk(token_pub_key,0),
    p2pk(bsv_pub_key,amount)
]

genesis_tx, response = spend_p2pkh(
    txs=token_funding_tx,
    indices=token_funding_ixs,
    outputs=outputs,
    index_output=1,
    public_keys=[token_pub_key]*len(token_funding_tx),
    fee_rate=10,
    network=connection
)

In [ ]:
print(f"Genesis txid: {genesis_tx.id()}")

In [ ]:
bsv_funding_tx = [
    tx_from_id(txid, connection) for txid in [
        "",
        "",
    ]
]

bsv_funding_ixs = [0,0]

In [ ]:
# Redistribute the funds allocated to bsv_pub_key
amount = sum([tx.tx_outs[index].amount for (tx, index) in zip(bsv_funding_tx, bsv_funding_ixs)])

outputs = [
    p2pk(bsv_pub_key,amount//2),
    p2pk(token_pub_key,amount//2),
]

zkp_funding, response = spend_p2pkh(
    txs=bsv_funding_tx,
    indices=bsv_funding_ixs,
    outputs=outputs,
    index_output=1,
    public_keys=[bsv_pub_key,bsv_pub_key],
    fee_rate=10,
    network=connection
)

bsv_zkp_funding_id = zkp_funding.id()
token_zkp_funding_id = zkp_funding.id()

print(f"ZKP funding: {zkp_funding.id()}")

In [ ]:
genesis_txid = genesis_tx.id()
print(f"Genesis txid: {genesis_tx.id()}")

bsv_zkp_funding = zkp_funding
bsv_zkp_funding_ix = 0
bsv_zkp_funding_id = zkp_funding.id()

token_zkp_funding = zkp_funding
token_zkp_funding_ix = 1
token_zkp_funding_id = zkp_funding.id()

print(f"ZKP funding: {zkp_funding.id()}")

### Initiate transfers

#### Transfer genesis

First of all, we transfer the genesis. In this case, we don't need any zk-verifier, so we simply execute the swap.

In [ ]:
# No need for a zk-verifier to transfer the genesis
swap_tx, response = execute_swap(
    token_txid=genesis_txid,
    bsv_txid=genesis_txid,
    bsv_index=1,
    token_pub_key=token_pub_key,
    bsv_pub_key=bsv_pub_key,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"

latest_transfer_txid = swap_tx.id()
latest_transfer_tx = swap_tx

print(f"Latest transfer txid: {latest_transfer_txid}")

Now we exchange the roles of `token_pub_key` and `bsv_pub_key`.

In [ ]:
token_pub_key, bsv_pub_key = bsv_pub_key, token_pub_key
bsv_index = 1

bsv_zkp_funding, token_zkp_funding = token_zkp_funding, bsv_zkp_funding
bsv_zkp_funding_ix, token_zkp_funding_ix = token_zkp_funding_ix, bsv_zkp_funding_ix

Execute the cell below to get the txid of the latest transaction containing the NFT.

In [ ]:
latest_transfer_tx.serialize().hex()

#### Transfer token

Now we transfer the token again. To create the zk-verifier below, we need to process the input to the PCD. Follow the steps below:

- Copy `genesis_txid` into `./configs/config_base/prove.toml` both as `genesis_txid` and as `outpoint_txid`
- Generate a proof for the base case (transfer of the genesis). The command is (run from `nft_proof_system`, see also the README for the Rust crate for more information)

    `cargo run --release -- --prove --file ./configs/config_base/prove.toml`
- Copy `genesis_txid` into `./configs/config_rec_1/prove.toml`
- Copy `latest_transfer_txid` as `outpoint_txid` into `./configs/config_rec_1/prove.toml`
- Copy `latest_transfer_tx` as `tx` into `./configs/config_rec_1/prove.toml`
- Process the input by executing the command

    `cargo run --release -- --process --file ./configs/config_rec_1/prove.toml`

Then, execute the cell below to deploy the zk-verifier

In [ ]:
# Deploy zk-verifier
bsv_zkp_funding, response = p2pk_to_groth16(
    tx=bsv_zkp_funding,
    index=bsv_zkp_funding_ix,
    path_input=processed_input_path + "proof_recursive_first_step_processed_input.bin",
    path_vk=vk_path,
    public_key=bsv_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
bsv_zkp_funding_ix = 1

groth16_verifier = bsv_zkp_funding
print(f"ZKP verifier: {groth16_verifier.id()}")

Now we generate a proof to deploy against the above verifier. We do so by executing the following command (see also the README in `nft_proof_system`):

`cargo run --release -- --prove --file ./configs/config_rec_1/prove.toml`

Then, execute the following cell to deploy the proof.

In [ ]:
# Deploy zk-proof
token_zkp_funding, response = spend_zkp_to_output(
    tx=groth16_verifier,
    index=0,
    funding_tx=token_zkp_funding,
    funding_index=token_zkp_funding_ix,
    path_proof=proof_path + "proof_recursive_first_step.bin",
    path_input=processed_input_path + "proof_recursive_first_step_processed_input.bin",
    path_vk=vk_path,
    public_key=token_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
token_zkp_funding_ix = 0

groth16_proof = token_zkp_funding
print(f"ZKP proof: {groth16_proof.id()}")

Now that the proof has been validated by the miners, we can proceed to publish the transaction transferring the token.

In [ ]:
# Execute swap
swap_tx, response = execute_swap(
    token_txid=latest_transfer_tx.id(),
    bsv_txid=latest_transfer_tx.id(),
    bsv_index=bsv_index,
    token_pub_key=token_pub_key,
    bsv_pub_key=bsv_pub_key,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"

latest_transfer_txid = swap_tx.id()
latest_transfer_tx = swap_tx

print(f"Latest transfer txid: {latest_transfer_txid}")

Finally, we exchange again the roles of `token_pub_key` and `bsv_pub_key`.

In [ ]:
token_pub_key, bsv_pub_key = bsv_pub_key, token_pub_key
bsv_index = 1

bsv_zkp_funding, token_zkp_funding = token_zkp_funding, bsv_zkp_funding
bsv_zkp_funding_ix, token_zkp_funding_ix = token_zkp_funding_ix, bsv_zkp_funding_ix

In [ ]:
latest_transfer_tx.serialize().hex()

#### Transfer token

To execute another swap, we proceed as in the previous case:

- Copy `genesis_txid` into `./configs/config_rec_2/prove.toml`
- Copy `latest_transfer_txid` as `outpoint_txid` into `./configs/config_rec_2/prove.toml`
- Copy `latest_transfer_tx` as `tx` into `./configs/config_rec_2/prove.toml`
- Process the input by executing the command `cargo run --release -- --process --file ./configs/config_rec_2/prove.toml`
- Execute the cell below to deploy the zk-verifier

In [ ]:
# Deploy zk-verifier
bsv_zkp_funding, response = p2pk_to_groth16(
    tx=bsv_zkp_funding,
    index=bsv_zkp_funding_ix,
    path_input=processed_input_path + "proof_recursive_second_step_processed_input.bin",
    path_vk=vk_path,
    public_key=bsv_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
bsv_zkp_funding_ix = 1

groth16_verifier = bsv_zkp_funding
print(f"ZKP verifier: {groth16_verifier.id()}")

Once again, we generate a proof to deploy against the above verifier. We do so by executing the following command:

`cargo run --release -- --prove --file ./configs/config_rec_2/prove.toml`

Then, execute the following cell to deploy the proof.

In [ ]:
# Deploy zk-proof
token_zkp_funding, response = spend_zkp_to_output(
    tx=groth16_verifier,
    index=0,
    funding_tx=token_zkp_funding,
    funding_index=token_zkp_funding_ix,
    path_proof=proof_path + "proof_recursive_second_step.bin",
    path_input=processed_input_path + "proof_recursive_second_step_processed_input.bin",
    path_vk=vk_path,
    public_key=token_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
token_zkp_funding_ix = 0

groth16_proof = token_zkp_funding
print(f"ZKP proof: {groth16_proof.id()}")

Execute the swap

In [ ]:
# Execute swap
swap_tx, response = execute_swap(
    token_txid=latest_transfer_tx.id(),
    bsv_txid=latest_transfer_tx.id(),
    bsv_index=bsv_index,
    token_pub_key=token_pub_key,
    bsv_pub_key=bsv_pub_key,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"

latest_transfer_txid = swap_tx.id()
latest_transfer_tx = swap_tx

print(f"Latest transfer txid: {latest_transfer_txid}")

Swap the roles of `token_pub_key` and `bsv_pub_key`.

In [ ]:
token_pub_key, bsv_pub_key = bsv_pub_key, token_pub_key
bsv_index = 1

bsv_zkp_funding, token_zkp_funding = token_zkp_funding, bsv_zkp_funding
bsv_zkp_funding_ix, token_zkp_funding_ix = token_zkp_funding_ix, bsv_zkp_funding_ix

In [ ]:
latest_transfer_tx.serialize().hex()

#### Transfer token

In [ ]:
# Deploy zk-verifier
bsv_zkp_funding, response = p2pk_to_groth16(
    tx=bsv_zkp_funding,
    index=bsv_zkp_funding_ix,
    path_input=processed_input_path + "proof_recursive_third_step_processed_input.bin",
    path_vk=vk_path,
    public_key=bsv_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
bsv_zkp_funding_ix = 1

groth16_verifier = bsv_zkp_funding
print(f"ZKP verifier: {groth16_verifier.id()}")

In [ ]:
# Deploy zk-proof
token_zkp_funding, response = spend_zkp_to_output(
    tx=groth16_verifier,
    index=0,
    funding_tx=token_zkp_funding,
    funding_index=token_zkp_funding_ix,
    path_proof=proof_path + "proof_recursive_third_step.bin",
    path_input=processed_input_path + "proof_recursive_third_step_processed_input.bin",
    path_vk=vk_path,
    public_key=token_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
token_zkp_funding_ix = 0

groth16_proof = token_zkp_funding
print(f"ZKP proof: {groth16_proof.id()}")

In [ ]:
# Execute swap
swap_tx, response = execute_swap(
    token_txid=latest_transfer_tx.id(),
    bsv_txid=latest_transfer_tx.id(),
    bsv_index=bsv_index,
    token_pub_key=token_pub_key,
    bsv_pub_key=bsv_pub_key,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"

latest_transfer_txid = swap_tx.id()
latest_transfer_tx = swap_tx

print(f"Latest transfer txid: {latest_transfer_txid}")

In [ ]:
token_pub_key, bsv_pub_key = bsv_pub_key, token_pub_key
bsv_index = 1

bsv_zkp_funding, token_zkp_funding = token_zkp_funding, bsv_zkp_funding
bsv_zkp_funding_ix, token_zkp_funding_ix = token_zkp_funding_ix, bsv_zkp_funding_ix

In [ ]:
latest_transfer_tx.serialize().hex()

#### Transfer token

In [ ]:
# Deploy zk-verifier
bsv_zkp_funding, response = p2pk_to_groth16(
    tx=bsv_zkp_funding,
    index=bsv_zkp_funding_ix,
    path_input=processed_input_path + "proof_recursive_fourth_step_processed_input.bin",
    path_vk=vk_path,
    public_key=bsv_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
bsv_zkp_funding_ix = 1

groth16_verifier = bsv_zkp_funding
print(f"ZKP verifier: {groth16_verifier.id()}")

In [ ]:
# Deploy zk-proof
token_zkp_funding, response = spend_zkp_to_output(
    tx=groth16_verifier,
    index=0,
    funding_tx=token_zkp_funding,
    funding_index=token_zkp_funding_ix,
    path_proof=proof_path + "proof_recursive_fourth_step.bin",
    path_input=processed_input_path + "proof_recursive_fourth_step_processed_input.bin",
    path_vk=vk_path,
    public_key=token_pub_key,
    fee_rate=20,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"
token_zkp_funding_ix = 0

groth16_proof = token_zkp_funding
print(f"ZKP proof: {groth16_proof.id()}")

In [ ]:
# Execute swap
swap_tx, response = execute_swap(
    token_txid=latest_transfer_tx.id(),
    bsv_txid=latest_transfer_tx.id(),
    bsv_index=bsv_index,
    token_pub_key=token_pub_key,
    bsv_pub_key=bsv_pub_key,
    network=connection,
)

assert response.status_code == 200, f"Swap failed, error is: {response.content}"

latest_transfer_txid = swap_tx.id()
latest_transfer_tx = swap_tx

print(f"Latest transfer txid: {latest_transfer_txid}")

In [ ]:
token_pub_key, bsv_pub_key = bsv_pub_key, token_pub_key
bsv_index = 1

bsv_zkp_funding, token_zkp_funding = token_zkp_funding, bsv_zkp_funding
bsv_zkp_funding_ix, token_zkp_funding_ix = token_zkp_funding_ix, bsv_zkp_funding_ix

In [ ]:
latest_transfer_tx.serialize().hex()